In [1]:
import torch
import torch.nn as nn
from diffusers.schedulers import PNDMScheduler
from pathlib import Path
from diffusers import DiffusionPipeline
from util import onnx_export
import numpy as np
from onnxruntime.quantization.quantize import quantize_dynamic
from onnxruntime.quantization import QuantType
import gc
gc.collect()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0

In [2]:
device = "cpu"
dtype = torch.float32

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float32)

---

## Decoder

In [4]:
class Decoder(nn.Module):
    def __init__(self, vae_decoder, device='cpu'):
        super().__init__()
        self.vae_decoder = vae_decoder
        self.vae_decoder = vae_decoder.to(device = device)
        self.device = device
  
    
    def denormalize(self, images):
        """
        Denormalize an image array to [0,1].
        """
        return (images / 2 + 0.5).clamp(0, 1)   

    def forward(self, latent_sample):
        # 0. preprocessing
        latent_sample = latent_sample / 0.18215 
        # 1. run
        image = self.vae_decoder(latent_sample)['sample']       # [1, 3, 512, 512]
        
        # 2. postprocessing
        image = self.denormalize(image).permute(0, 2, 3, 1)
        #image = self.pt_to_numpy(image)
         
        return image #self.numpy_to_pil(image)

In [5]:
vae_decoder = pipeline.vae
vae_decoder.forward = pipeline.vae.decode
decoder = Decoder(vae_decoder, device)

# decoder(torch.randn(1, 4, 64, 64).to(device=device, dtype=dtype))
onnx_export(
    decoder,
    model_args=(
        torch.randn(1, 4, 64, 64).to(device=device, dtype=dtype)
    ),
    output_path = Path('../onnx-models/Decoder.onnx'),
    ordered_input_names = ["latent_sample"],   # 입력값 : latent_sample
    output_names = ["output"],
    dynamic_axes = {
            "latent_sample": {0: "batch", 1: "channels", 2: "height", 3: "width"},
        },
    opset=14,
)
quantize_dynamic(
    model_input = f'../onnx-models/Decoder.onnx',
    model_output=f'../onnx-models/Decoder-quant.onnx',
    per_channel=False,
    reduce_range=False,
    weight_type=QuantType.QUInt8,
)

ONNX export Start🚗


/opt/conda/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/opt/conda/lib/python3.10/site-packages/torch/onnx/utils.py:687: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/opt/conda/lib/python3.10/site-packages/torch/onnx/utils.py:1178: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the export

ONNX export Finish🍷
